# Using NCO utilities ncks and ncrcat to subset CETB files by location and concatenate by time series

To begin, you need to know the map coordinates of the spatial subset you want.

You can use various utilities to do this, we have written the python package cetbtools.ease2conv to help with it.

For this example, I want the upper left quadrant of the EASE2_N25km grid, which is bounded by
the upper left corner of the grid and the North Pole at the center of the grid.

# There are Three Stages of Files: 
## - Web Scraped 
    - These are raw netCDF files with much more information than just Tb data 
## - Subsetted 
    - These are geographically subsetted to the North Slope of Alaska and only contain arrays of Tb information 
## - Concatenated
    - These are essentially time cubes of Tb matrices stored in netCDF files (contain entire year)
## - LongFile
    - Consists of many years concatenated together, this is a complete dataset 

In [ ]:
#%cd /Users/williamnorris/SWE_TB/data

In [1]:
%pylab notebook
import netCDF4
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
from cetbtools.ease2conv import Ease2Transform
import subprocess
from subprocess import call
import shlex
import os
import sys
import glob

Populating the interactive namespace from numpy and matplotlib


In [2]:
path = os.getcwd()
path

'/Users/williamnorris/SWE_TB'

## Get Coordinates

#### Convert lat/lon coordinates to EASE Grid 2.0 coordinates 
- Important to remember EASE Grid is oriented from the Pole, so it will look inverted 
- This subset includes the entirety of the brooks range and the North Slope 
![title](North_Slope_Subset.png)

In [3]:
# Upper Left of North slope 
N6 = Ease2Transform("EASE2_N6.25km")
lat, lon = 62.27, -140.17
row, col = N6.geographic_to_grid(lat, lon)
x, y = N6.grid_to_map(row, col)
print row,col
print x, y

1062.91625534 1125.40805254
-1963074.67159 2353648.4041


In [4]:
# Lower right of North Slope 
lat, lon =73.64, -166.08
row, col = N6.geographic_to_grid(lat, lon)
x, y = N6.grid_to_map(row, col)
print row,col
print x, y

1156.75516773 1369.42297793
-437981.387918 1767155.20168


So these min/max values for x and y can be passed to ncks to subset the TB variable from a CETB file with:

cmd = '/anaconda/envs/ipykernel_py2/bin/ncks -d x,-1963074.67,-437981.38 -d y,1767155.2,2353648.4 -v TB ' + infile + " " + outfile

## Make List of Files

In [20]:
#%cd wget
os.chdir(path + '/data/wget')

In [21]:
# Make a list of the files to concatenate together for 19H
list19 = sort(glob.glob("*19H-M-SIR*"))

# Make list for the 37GHz
list37 = sort(glob.glob("*37H-M-SIR*"))

## Subset Files to Geo Location 
### Enter paths of file locations and destinations

In [22]:
os.chdir(path)
wget = path + "/data/wget/"
#wget37 = path + "/37GHz_Wget/"
path19 = path + "/data/Subsetted_19H/"
path37 = path + "/data/Subsetted_37H/"

In [24]:
# for loop to take each file in the list and subset it's location 
for file in list19:
    outfile = path19 + file
    infile = wget + file
    cmd = '/anaconda3/envs/python2/bin/ncks -d x,-1963074.67,-437981.38 -d y,1767155.2,2353648.4 -v TB ' + infile + " " + outfile
    print("Next in : %s" % file)
    print("Next out: %s" % outfile)
    # print("next unl out: %s" % unlfile)
    subprocess.call(shlex.split(cmd), shell = False)
    os.remove(infile)
    print("Done")

    
for file in list37: 
    outfile = path37 + file 
    infile = wget + file
    cmd = '/anaconda3/envs/python2/bin/ncks -d x,-1963074.67,-437981.38 -d y,1767155.2,2353648.4 -v TB ' + infile + " " + outfile
    print("Next in: %s" % infile)
    print("Next out: %s" % outfile)
    subprocess.call(shlex.split(cmd), shell = False)
    os.remove(infile)
    print("Done")   

Next in : NSIDC-0630-EASE2_N6.25km-F19_SSMIS-2015001-19H-M-SIR-CSU-v1.2.nc
Next out: /Users/williamnorris/SWE_TB/data/Subsetted_19H/NSIDC-0630-EASE2_N6.25km-F19_SSMIS-2015001-19H-M-SIR-CSU-v1.2.nc
Done
Next in : NSIDC-0630-EASE2_N6.25km-F19_SSMIS-2015002-19H-M-SIR-CSU-v1.2.nc
Next out: /Users/williamnorris/SWE_TB/data/Subsetted_19H/NSIDC-0630-EASE2_N6.25km-F19_SSMIS-2015002-19H-M-SIR-CSU-v1.2.nc
Done
Next in : NSIDC-0630-EASE2_N6.25km-F19_SSMIS-2015003-19H-M-SIR-CSU-v1.2.nc
Next out: /Users/williamnorris/SWE_TB/data/Subsetted_19H/NSIDC-0630-EASE2_N6.25km-F19_SSMIS-2015003-19H-M-SIR-CSU-v1.2.nc
Done
Next in : NSIDC-0630-EASE2_N6.25km-F19_SSMIS-2015004-19H-M-SIR-CSU-v1.2.nc
Next out: /Users/williamnorris/SWE_TB/data/Subsetted_19H/NSIDC-0630-EASE2_N6.25km-F19_SSMIS-2015004-19H-M-SIR-CSU-v1.2.nc
Done
Next in : NSIDC-0630-EASE2_N6.25km-F19_SSMIS-2015005-19H-M-SIR-CSU-v1.2.nc
Next out: /Users/williamnorris/SWE_TB/data/Subsetted_19H/NSIDC-0630-EASE2_N6.25km-F19_SSMIS-2015005-19H-M-SIR-CSU-v1.

## Concatenate each year of files together 

### Keep in mind the 19H files are 6.25km pixels and the 37H are 3.25km pixels

In [25]:
os.chdir(path)
os.chdir(path + '/data' + '/Subsetted_19H')
#%cd Subsetted_19H
# Concatenate 19GHz files:
!/anaconda3/envs/python2/bin/ncrcat NSIDC*nc -O all_days_2015_19H_F19.nc
#%cd ..

os.chdir(path + '/data' + '/Subsetted_37H')
#%cd Subsetted_37H
# Concatenate 37GHz files: 
!/anaconda3/envs/python2/bin/ncrcat NSIDC*nc -O all_days_2015_37H_F19.nc
#%cd ..

ncrcat: INFO/WARNING Multi-file concatenator encountered packing attribute scale_factor for variable TB. NCO copies the packing attributes from the first file to the output file. The packing attributes from the remaining files must match exactly those in the first file or data from subsequent files will not unpack correctly. Be sure all input files share the same packing attributes. If in doubt, unpack (with ncpdq -U) the input files, then concatenate them, then pack the result (with ncpdq). This message is printed only once per invocation.
ncrcat: INFO/WARNING Multi-file concatenator encountered packing attribute scale_factor for variable TB. NCO copies the packing attributes from the first file to the output file. The packing attributes from the remaining files must match exactly those in the first file or data from subsequent files will not unpack correctly. Be sure all input files share the same packing attributes. If in doubt, unpack (with ncpdq -U) the input files, then concatena

In [26]:
# To remove all the excess files in the directory 
# Only use when done with individual files!!!
os.chdir(path + '/data' + '/Subsetted_19H')
#%cd Subsetted_19H
filelist = glob.glob('NSIDC*')
for f in filelist:
    os.remove(f)
#%cd ..
    
os.chdir(path + '/data' + '/Subsetted_37H')
#%cd Subsetted_37H
filelist = glob.glob('NSIDC*')
for f in filelist:
    os.remove(f)
#%cd ..

## Make Long File Out of Each Year's File

In [ ]:
# Make Long File with multiple satellite sensors for 2000-2008
%cd /Users/williamnorris/SWE_Clean/19H_Concatenated
!/anaconda/envs/python2/bin/ncrcat all*nc -O /Users/williamnorris/SWE_Clean/LongFile/all_days_19H 

In [3]:
#%cd /Users/williamnorris/SWE_Clean/37H_Concatenated
!/anaconda3/envs/gis/bin/ncrcat all*nc -O /Users/williamnorris/SWE_Clean/LongFile/all_days_37H 

/bin/sh: /anaconda3/envs/gis/bin/ncrcat: No such file or directory
